### The Nested Clustered Optimization Algorithm

7장의 나머지 부분은 마코위츠의 저주의 근원을 다른 새로운 머신러닝 기반 방법인 중첩 군집 최적화($\text{NCO}$, Nested Clustered Optimization)를 도입하는 데 초점을 맞춘다. $\text{NCO}$는 ‘래퍼(wrapper)’라고 알려진 알고리즘의 종류에 속한다. 어떤 효율적 경계의 구성 원소가 계산되는지 여부나 어떤 제약 조건들이 부과되는지 여부에 대해서는 무관하다. $\text{NCO}$는 마코위츠의 저주가 기존의 평균–분산 배분 방법에 미치는 영향을 해결하기 위한 전략을 제공한다.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from FinancialMachineLearning.data_loader.tickers import WikipediaStockListing
from curl_cffi import requests

tickers = WikipediaStockListing()
sp100_tickers = tickers.sp100()['Symbol'].tolist()

session = requests.Session(
    impersonate = 'chrome'
)

data = yf.download(
    sp100_tickers,
    start = '2020-01-01',
    progress = False,
    auto_adjust = True,
    interval = '1d',
    session = session
)['Close'].resample('W-FRI').last()

ret = data.pct_change().iloc[1:]

#### 1. Correlation Clustering

우선 $\text{NCO}$ 알고리즘의 첫 번째 단계는 상관 행렬을 군집화하는 것이다. 이 작업에는 최적의 군집 수를 찾는 작업이 포함된다. 한 가지 가능성은 $\text{ONC}$ 알고리즘(Chapter 4)을 적용하는 것이지만, $\text{NCO}$는 군집 수를 결정하는 데 사용되는 어떤 알고리즘이 사용되는가에 대해서는 무관하다. $T/N$이 상대적으로 낮은 큰 행렬의 경우 2장에서 설명한 방법에 따라 군집화 전에 상관 행렬의 잡음을 제거하는 것이 바람직하다. 아래 코드는 이 절차를 구현한다. Chapter 2에서 소개한 `deNoiseCov` 함수를 사용해 잡음이 제거된 공분산 행렬(`cov1`)을 계산한다. 주의 사항으로 인수 $q$는 관측 행렬의 행 수와 열 수 사이의 비율을 알려 준다. `bWidth = 0`일 때 공분산 행렬의 잡음은 제거되지 않는다. `cov2corr` 함수를 사용해 결과 공분산 행렬을 상관 행렬로 표준화한다. 그런 다음 Chapter 4에서 소개한 `clusterKMeansBase` 함수를 사용해 정제된 상관 행렬을 군집화한다. `maxNumClusters` 인수는 상관 행렬의 열 수의 절반으로 설정된다. 그 이유는 단일 항목 군집은 행렬의 조건 수 증가를 유발하지 않기 때문에 최소 크기가 2인 군집만 고려하면 되기 때문이다. 군집 수가 더 작을 것으로 예상되는 경우 더 낮은 `maxNumClusters`를 사용해 계산을 가속화할 수 있다.


In [2]:
from FinancialMachineLearning.filter.denoising import denoise_covariance

cols = ret.cov().columns # covariance
cov1 = denoise_covariance(
    ret.cov(), 
    q = len(ret) / len(ret.columns), 
    b_width = 0.01
)

In [3]:
cov1 = pd.DataFrame(cov1, index = cols, columns = cols)

In [4]:
from FinancialMachineLearning.filter.denoising import covariance_to_correlation
from FinancialMachineLearning.machine_learning.clustering import clusterKMeansBase

corr1 = covariance_to_correlation(cov1)
corr2, clusters, silh = clusterKMeansBase(
    corr1,
    maxNumClusters = ret.corr().shape[0]/2,
    n_init = 10
)

일반적인 질문은 `corr1` 또는 `corr1.abs()`를 군집화해야 하는가다. 모든 상관관계가 음수가 아닌 경우 `corr1`과 `corr1.abs()` 군집화는 동일한 결과를 산출한다. 일부 상관관계가 음수일 경우 답은 더 복잡하며, 관측된 입력의 수치 특성에 따라 달라진다. 두 가지를 모두 시도해 보고, 몬테카를로 실험에서 어떤 군집화가 특정 `corr1`에 더 잘 작용하는지 볼 것을 추천한다.

#### 2. Intracluster Weights

NCO 알고리즘의 두 번째 단계는 잡음이 제거된 공분산 행렬, `cov1`을 사용해 최적의 군집 내 배분을 계산하는 것이다. 아래 코드는 이 절차를 구현한다. 단순성을 위해 `optimizing_portfolio` 함수에 구현된 최소 분산 배분을 기본으로 설정했다. 그러나 이 절차는 다른 대체적인 배분 방법의 사용도 허용한다. 측정된 군집 내 가중치를 사용해 군집 간 상관관계를 나타내는 축약된 공분산 행렬인 `cov2`를 도출할 수 있다.

In [5]:
weight_intra = pd.DataFrame(
    0, index = ret.cov().index,
    columns = clusters.keys()
)

In [6]:
weight_intra # 군집 내에서의 최적 배분

,0,1,2
Ticker,,,
AAPL,0,0,0
ABBV,0,0,0
ABT,0,0,0
ACN,0,0,0
ADBE,0,0,0
...,...,...,...
V,0,0,0
VZ,0,0,0
WFC,0,0,0


In [7]:
from FinancialMachineLearning.utils.stats import optimizing_portfolio
import warnings
warnings.filterwarnings('ignore')

for i in clusters :
    weight_intra.loc[clusters[i], i] = optimizing_portfolio(
        cov1.loc[clusters[i], clusters[i]]
    ).flatten()
cov2 = weight_intra.T.dot(np.dot(cov1, weight_intra))

In [8]:
cov2 # 군집 내 최적 비분의 축약된 공분산 행렬

,0,1,2
0,0.000503,0.000232,0.000216
1,0.000232,0.000405,0.000223
2,0.000216,0.000223,0.000370


#### 3. Intercluster Weights

NCO 알고리즘의 세 번째 단계는 축약된 공분산 행렬인 cov2를 사용해 최적의 군집 간 배분을 계산하는 것이다. 구상의 이 공분산 행렬은 대각 행렬에 가깝고, 최적화 문제는 이상적인 마코위츠 사례에 가깝다. 즉 군집화와 군집 내 최적화 단계를 통해 ‘마코위츠–저주(Markowitz-cursed)’ 문제($|\rho| > 0$)를 올바른 작동하는 문제($\rho \sim 0$)으로 전환할 수 있다.

아래 코드는 이 절차를 구현한다. 군집 내 배분 단계의 경우 `optimizing_portfolio` 함수에서 사용된 것과 동일한 배분 절차를 적용한다. 증권별 최종 배분은 `final_weight` 데이터 프레임에 의해 표현되며, 이는 군집 내 비중과 군집 간 비중을 곱한 결과로 나타난다.

In [9]:
weight_inter = pd.Series(
    optimizing_portfolio(cov2).flatten(),
    index = cov2.index
)

In [10]:
weight_inter # 군집 간 배분 비중

0    0.226402
1    0.334619
2    0.438979
dtype: float64

In [11]:
final_weight = weight_intra.mul(
    weight_inter, axis = 1
).sum(axis = 1).sort_index()

In [12]:
final_weight

Ticker
AAPL    0.009663
ABBV   -0.014838
ABT    -0.002285
ACN     0.019978
ADBE   -0.005668
          ...   
V       0.042866
VZ      0.062604
WFC     0.006117
WMT     0.067304
XOM     0.021907
Length: 101, dtype: float64

### Experimental Results

이번 절에서는 NCO 알고리즘을 통계된 실험에 적용하고, 그 성능을 마코위츠의 접근법과 비교한다. 2장에서와 마찬가지로 제약 없는 효율적 프론티어의 어떤 구성 포트폴리오도 효율적 프론티어의 두 가지 특성 포트폴리오, 즉 최소 분산 포트폴리오와 최대 샤프 비율 포트폴리오의 블록 조합으로 도출될 수 있기 때문에(분리 정리 separation theorem으로 알려져 있는 결과) 이들에 대해 논의한다.

아래 코드는 앞서 소개한 NCO 알고리즘을 구현한다. 인수 `mu`가 `None`이면 최소 분산 포트폴리오를 반환하는 반면, `mu`가 `None`이 아닌 경우 `optPort_nco` 함수는 최대 샤프 비율 포트폴리오를 반환한다.

In [13]:
pd.DataFrame(ret.mean())

,0
Ticker,
AAPL,0.004600
ABBV,0.004179
ABT,0.002451
ACN,0.002072
ADBE,0.001613
...,...
V,0.002885
VZ,0.000259
WFC,0.003469


In [14]:
from FinancialMachineLearning.machine_learning.clustering import nested_clustered_optimization

nco = nested_clustered_optimization(
    cov = cov1, # empirical Covariance
    mu = None,
    maxNumClusters = 10
)

#### 1. Minimum Variance Portfolio

아래의 코드는 50개 증권 포트폴리오의 전형적 버전을 나타내는 랜덤 평균 벡터와 랜덤 공분산 행렬을 생성하며, 군집 내 상관관계가 0.5인 10개 블록으로 그룹화한다. 이 벡터와 행렬은 관측치를 생성하는 ‘실제’ 프로세스의 특성을 나타낸다. 다른 파라미터를 사용해 여러 시행 결과를 재현하고 비교하기 위한 시드(seed)를 설정했다. 함수 `formTrueMatrix`는 2장에서 선언됐다.

In [15]:
mu0 = np.array([ret.mean().values]).T
cov0 = ret.cov().values

아래의 코드는 `simCovMu` 함수를 사용해 실제 프로세스에서 도출된 1,000개의 관측치를 기반으로 랜덤 경험적 평균 벡터와 랜덤 경험적 공분산 행렬을 시뮬레이션한다(2장에 명시됨). `shrink=True`일 때 경험적 공분산 행렬은 레드와-울프의 축소 방법을 따른다. 경험적 공분산 행렬을 사용해 함수 `optPort`(2장에서도 선언됨)는 마코위츠에 따른 최소 분산 포트폴리오를 추정하며, 함수 `optPort_nco`는 $\text{NCO}$ 알고리즘을 적용해 최소 분산 포트폴리오를 추정한다. 이 절차는 1,000개의 서로 다른 랜덤 경험적 공분산 행렬에 반복된다. `minVarPortf=True`이면 랜덤 경험적 평균 벡터는 사용되지 않는다는 점을 주목하자.

In [17]:
from tqdm import tqdm

number_of_simulation = 100

mu1 = ret.mean()
w1 = pd.Series(dtype = float)
w1_d = pd.Series(dtype = float)

for i in tqdm(range(number_of_simulation)) : # 시간복잡도는 O(100^3)이므로 과부하에 주의
    w1.loc[i] = optimizing_portfolio(cov1, mu1).flatten()
    w1_d.loc[i] = nested_clustered_optimization(
        cov1, 
        mu = mu1, 
        maxNumClusters = int(ret.cov().shape[0] / 2)
    ).flatten()

100%|██████████| 100/100 [11:44<00:00,  7.04s/it]


아래의 코드는 실제 공분산 행렬에서 파생된 실제 최소 분산 포트폴리오를 계산한다. 그런 배분을 벤치마크로 사용해 모든 비중에 대한 $\text{RMSE}$를 계산한다. 시뮬레이션에서 축소한 경우와 아닌 경우를 실행할 수 있으므로 표에 표시된 네 가지 조합을 얻을 수 있다.

In [46]:
w0 = optimizing_portfolio(ret.cov(), None)

In [47]:
w0 = np.repeat(w0.T,w1.shape[0], axis = 0)

In [48]:
rmsd = np.mean((np.array(w1.tolist()) - w0).flatten() ** 2) ** 0.5

In [49]:
rmsd_d = np.mean((np.array(w1_d.tolist()) - w0).flatten() ** 2) ** 0.5

In [50]:
print(f'RMSD using mean variance optimization : {rmsd :.4f}')
print(f'RMSD using nested clustered optimization : {rmsd_d :.4f}')

RMSD using mean variance optimization : 0.1604
RMSD using nested clustered optimization : 0.1451


||마코위츠|NCO|
|:---|:---:|:---:|
|원|7.95E-03|4.21E-03|
|축소|8.89E-03|6.74E-03|

$\text{NCO}$는 마코위츠 $\text{RMSE}$의 52.98%인 최소 분산 포트폴리오를 계산한다. 즉 $\text{RMSE}$의 47.02%가 감소한다. 르드와-울프 축소법은 $\text{RMSE}$를 감소시키는 데 도움을 주는데, 그 감소는 11.81% 정도로 상대적으로 적다. 축소와 $\text{NCO}$를 결합하면 $\text{RMSE}$가 15.30% 감소하는데, 이는 축소보다는 낮지만 $\text{NCO}$보다 못하다.

$\text{NCO}$가 마코위츠 해보다 훨씬 낮은 $\text{RMSE}$를 50개 증권으로 구성된 소규모 포트폴리오에서도 제공하고 있으며, 축소법은 아무런 가치가 더하지 않는다는 것을 시사한다. $\text{NCO}$의 장점은 더 큰 포트폴리오에서 더 커진다는 것을 테스트하는 것은 쉽다.

#### 2. Maximum Sharpe Ratio Portfolio

`minVarPortf=False`를 설정하고, 위 과정을 다시 실행해 최대 샤프 비율 포트폴리오와 관련된 $\text{RMSE}$를 도출할 수 있다. 아래의 표는 이 실험의 결과를 보고한다.

$\text{NCO}$는 마코위츠 $\text{RMSE}$의 45.17%로 최대 샤프 비율 포트폴리오를 계산한다. 즉 $\text{RMSE}$의 54.83%가 감소한다. 축소법과 $\text{NCO}$의 조합은 최대 샤프 비율 포트폴리오의 $\text{RMSE}$를 18.52% 감소하는데, 감소량이 축소법보다는 낮지만 $\text{NCO}$보다는 나쁘다. 다시 말하지만, $\text{NCO}$는 마코위츠의 해보다 훨씬 낮은 $\text{RMSE}$를 제공한다. 축소법은 거의 아무런 효과가 없다. 
$\text{NCO}$를 사용하도록 하자.

||   마코위츠   |   NCO    |
|:---|:--------:|:--------:|
|원| 7.02E-02 | 3.17E-02 |
|축소| 6.54E-02 | 5.72E-02 |


### Conclusions

마코위츠의 포트폴리오 최적화 프레임워크는 수학적으로 정확하지만, 그것의 실제 적용은 수치적인 문제로 어려움을 겪고 있다. 특히 금융 공분산 행렬은 잡음과 신호 때문에 조건 수가 높게 나타난다. 이러한 공분산 행렬의 역행렬은, 추정 오차를 확대해서 불안정한 해법으로 이어진다. 관측 행렬에서 몇 개의 행을 변경하면 완전히 다른 배분이 발생할 수 있다. 배분 추정기가 불편성을 갖더라도 이러한 불안정한 해와 관련된 분산은 큰 거래비용으로 이어져 수익성의 많은 부분을 없앨 수 있다.

7장에서는 마코위츠의 불안정 문제의 근원을 상관 행렬의 고유값 함수의 모양으로 추적해 봤다. 수평적 고유값 함수는 마코위츠의 프레임워크에 이상적이다. 증권 군집이 나뉘지 투자 유니버스에 비해 그들 사이에 더 큰 상관관계를 보이는 금융에서 고유값 함수는 수평이 아니며, 이는 다시 높은 조건 수의 원인이 된다. 잡음이 아니라 신호가 이런 유형의 공분산 불안정성의 원인이다.

최적화 문제를 군집당 하나의 최적화를 계산하고 모든 군집에 걸쳐 하나의 최종 최적화를 계산하는 식으로 몇 가지 문제로 나눠 이러한 불안정성을 다루고자 $\text{NCO}$ 알고리즘을 도입했다. 각 증권은 각각 하나의 군집에만 속하기 때문에 최종 배분은 군집 내와 군집 간 비중의 곱이다. 실험 결과는 이러한 이중 군집화 접근법이 마코위츠의 추정 오류를 현저하게 줄일 수 있다는 점을 보여 준다. $\text{NCO}$ 알고리즘은 유연하며 블랙–리터먼(Black–Litterman), 축소법, 역최적화 또는 제약 없는 최적화 접근법과 같은 다른 프레임워크와 결합해 활용할 수 있다. $\text{NCO}$를 일반 최적화 문제를 하위 문제로 나누는 전략으로 생각할 수 있으며, 이는 연구자가 선호하는 방법으로 해결할 수 있다.

다른 많은 머신러닝 알고리즘과 마찬가지로 $\text{NCO}$는 유연하고 모듈적이다. 예를 들어, 상관 행렬이 군집 내에 군집이 있는 강력한 계층 구조를 보이는 경우 행렬의 트리 같은 구조를 모방해 각 군집과 하위 군집 내에서 $\text{NCO}$ 알고리즘을 적용할 수 있다. 목표는 트리의 각 수준에서 수치적 불안정성을 억제해 하위 군집 내의 불안정성이 상위 군집 또는 나머지 상관 행렬로 확장되지 않도록 하는 것이다.

7장에 요약된 몬테카를로 접근 방식을 따라 특정 입력 변수 집합에 대한 다양한 최적화 방법에 의해 생성된 배분 오차를 추정할 수 있었다. 그 결과로 어떤 방법이 특정한 경우에 가장 강건한지에 대해 정확하게 결정할 수 있었다. 따라서 항상 하나의 특정 접근법에 의존하기보다는 특정 환경에 가장 적합한 최적화 방법을 기회적으로 적용할 수 있다.